In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"
import torch
import numpy as np
os.environ['TORCH'] = torch.__version__
os.environ['DGLBACKEND'] = "pytorch"
import dgl
import dgl.graphbolt as gb
import torch.nn as nn
import torch.nn.functional as F
device = torch.device("cpu")

Loading Dataset

In [ ]:
dataset = gb.BuiltinDataset("ogbn-arxiv").load()